In [2]:
import argparse
import json
import os
import sys
import traceback
from glob import glob
import math
import random
import numpy as np
from dotenv import load_dotenv
from azureml.core import Dataset, Datastore, Experiment, Run, Workspace
from azureml.core.authentication import AzureCliAuthentication
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import tensorflow.keras
from tensorflow.keras.layers import Flatten, Input, concatenate, Dense, Activation, Dropout, BatchNormalization,  MaxPooling2D, AveragePooling2D, Conv2D
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from azureml.core import Run
from numpy.random import seed
from tensorflow.random import set_seed
from dotnetcore2 import runtime
runtime.version = ("18", "10", "0")

2021-11-29 11:19:24.700393: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-29 11:19:24.700469: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
    #cli_auth = AzureCliAuthentication()

    #Connect to workspace
    ws = Workspace.from_config('config.json')

In [4]:
from azureml.core import Experiment
exp = Experiment(workspace=ws, name="imdb_train")

script_folder = os.path.join(os.getcwd(), "imdb_training")
os.makedirs(script_folder, exist_ok=True)

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_folder', help='Test data folder mounting point')
parser.add_argument('--epochs', type=str, dest='epochs', help='Amount of epochs to train')
parser.add_argument('--batch_size', type=str, dest='batch_size', help='Batch size')
parser.add_argument('--model_name', type=str, dest='model_name', help='Model name')
args = parser.parse_args()

data_folder = args.data_folder
print('Data folder:', data_folder)

dataset_train = np.load(os.path.join(data_folder, 'dataset_train.npy'))
dataset_test = np.load(os.path.join(data_folder, 'dataset_test.npy'))

run = Run.get_context()

batch_size = int(args.batch_size)
epochs = int(args.epochs)

In [ ]:
# Building the autoencoder
from tensorflow.keras import metrics

autoencoder = Sequential()
#Decode
autoencoder.add(InputLayer((3952,)))
autoencoder.add(Dense(1000, activation= 'relu' ))
#Bottleneck
autoencoder.add(Dense(120, activation= 'relu' ))
#Encode
autoencoder.add(Dense(1000, activation= 'relu' ))

autoencoder.add(Dense(3952, activation= 'sigmoid' ))
autoencoder.summary()




In [ ]:
# Training the autoencoder with a custom loss function

def custom_loss(y_true,y_pred):
    y_mask=tf.keras.backend.clip(y_true, 0, 0.01)*100
    return K.mean(K.square(y_mask*(y_pred - y_true)), axis=-1)

early_stopping_callback = tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
reduce_lr = tensorflow.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1)

autoencoder.compile(loss=custom_loss, optimizer='adam')
autoencoder.fit(np.array(dataset_train),
                np.array(dataset_train),
                validation_split=0.2,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                callbacks=[reduce_lr,early_stopping_callback])

In [ ]:
test_not_null = []
pred_not_null = []
predictions = autoencoder.predict(dataset_test)
for i in range(len(dataset_train)):
    indeces = np.nonzero(np.array(dataset_train.iloc[i-1]))
    test = []
    pred = []
    test_not_null.append(np.array([dataset_train.iloc[i-1,index] for index in indeces[0]], dtype=float))
    pred_not_null.append(np.array([predictions[i-1][index] for index in indeces[0]], dtype=float))



In [ ]:
THRESHOLD = 1
from sklearn.metrics import mean_squared_error
import math
means = []
for i in range(len(test_not_null)):
    means.append(np.mean(np.power(np.array(test_not_null[i]) - np.array(pred_not_null)[i],2)))
mse = np.mean(means)
print("MSE")
print(np.mean(mse))
print("STD")
variances=[]
for i in range(len(test_not_null)):
    n = len(test_not_null[i])
    mean = sum((np.array(test_not_null[i]))) / n
    deviations = [(x - mean) ** 2 for x in np.array(np.array(test_not_null[i]))]
    variance = sum(deviations) / n
    variances.append(variance)
    
std = math.sqrt(np.mean(variances))
print(std)
#Super Low std => Values don't lay far from apart => the review scores don't lay far apart